## <span style="color:pink">**PROJETO FINAL DE MACHINE LEARNING 2022.1**</span>
### **COLORIZAÇÃO DE IMAGENS BASEADA EM CAPAS DE ÁLBUNS**
NICOLE SARVASI ALVES DA COSTA & KAÍQUE DOGNANI

Importação de bibliotecas:

In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import img_to_array, load_img
from skimage.color import rgb2hsv
import numpy as np
import os
from skimage.color import hsv2rgb
import PIL
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Criação do arquivo .pkl

def RGB2X_Y():
    with open('dados.pkl', 'rb') as file:
        newDataSet = pickle.load(file)
        file.close()
    newDataSet = newDataSet[np.random.choice(newDataSet.shape[0], 5000), :, :, :]
    newDataSet = newDataSet/255.0
    for index, img in enumerate(newDataSet):
        newDataSet[index] = rgb2hsv(img)
    X_hsv = np.expand_dims(newDataSet[:,:,:,2], 3)
    Y_hsv = newDataSet[:,:,:,:2]

    return X_hsv, Y_hsv

X_train, Y_train = RGB2X_Y()

with open('dados_XY.pkl', 'wb') as file:
    pickle.dump((X_train, Y_train), file)

In [ ]:
with open('dados_XY.pkl', 'rb') as file:
    X_train, Y_train = pickle.load(file)

### Pegando o arquivo de treino e diminuindo o número de imagens

In [ ]:
X_train = X_train[:3500]
Y_train = Y_train[:3500]
X_train.shape

## Definição do nosso modelo

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, input_shape=[150,150,1], kernel_size=3, padding="same", activation="relu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(16, kernel_size=3, padding="same", activation="selu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(8, kernel_size=3, padding="same", activation="linear"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(16, kernel_size=3, padding="same", activation="selu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="selu"),
    keras.layers.UpSampling2D(size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Resizing(150, 150),
    keras.layers.Conv2D(2, kernel_size=3, padding="same", activation="linear"), 
])

## Compilando o modelo e plotando informações sobre ele

In [ ]:
model.compile(loss="mse", optimizer="nadam")

In [ ]:
model.summary()

## Treinando o modelo com uma função de EarlyStopping que para o treinamento quando chegamos nas medidas estabelecidas e para evitar que percamos qualquer evolução do modelo

In [ ]:
callback = keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0.0001,
    patience=30,
    mode="min",
    restore_best_weights=True,
)

history = model.fit(X_train, Y_train, epochs=1000, batch_size=15, callbacks=[callback])

## Aqui salvamos o modelo para futuramente podermos utilizá-lo

In [ ]:
model.save('MyModel_2')

## Abertura do modelo já salvo

In [ ]:
model = keras.models.load_model("MyModel")

In [ ]:
model.summary()

## Realizamos esta etapa para ter certeza de que as imagens que entram no preditor tenham 150X150

In [ ]:
import PIL
path = "C:\\Users\\Acer\\Pictures\\paramore.jpg"
file = PIL.Image.open(path)
if file.size != (150,150):
    file = file.resize((150,150))
    file.save(path)
    
img = img_to_array(load_img(path))

## Transformação da imagem de RGB para HSV e divisão entre img_X que são as componentes H e S, e img_Y que é a componente V

In [ ]:
img_hsv   = rgb2hsv(img/255.0)
img_X_test = np.expand_dims(img_hsv[:,:,2],2)
img_Y_test = img_hsv[:,:,:2]

## Rodamos o modelo com a imagem de teste

In [ ]:
img_Y_pred = model.predict(np.array([img_X_test]))
img_Y_pred.shape

## Passamos a imagem de HSV para RGB

In [ ]:
img_pred_hsv = np.concatenate((img_Y_pred[0,:,:,:],img_X_test), axis=2)
img_test_hsv = np.concatenate((img_Y_test,img_X_test), axis=2)

In [ ]:
print("img_teste_hsv",img_test_hsv.shape)
print("img_pred_hsv",img_pred_hsv.shape)

In [ ]:
Imagem_Test = (255*hsv2rgb(img_test_hsv)).astype(np.uint8)
Imagem_Pred = (255*hsv2rgb(img_pred_hsv)).astype(np.uint8)
print("Imagem_Pred",Imagem_Pred.shape)
print("Imagem_Test",Imagem_Test.shape)

## E finalmente plotamos a imagem final

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.add_subplot(1,2,1)
plt.imshow(Imagem_Test)
plt.axis('off')
plt.title("Test")

fig.add_subplot(1,2,2)
plt.imshow(Imagem_Pred)
plt.axis('off')
plt.title("Pred")